# Linear Regression Modeling of King County Real Estate Sale Prices


<figure>
    <img src="Illustrations/Seattle-sky.jpeg"
         align="Center"
         width="600"
         height="300">
</figure>

## Overview

 - Use multiple linear regression modeling to analyze house sales in a King County, Washington.

## Business Problem

## Data Understanding

## Data Preparation

## Linear Regression Modeling

## Data Analysis and Recommendation

## Evaluation

## Conclusion